# Data Driven Business project

-
-
-
- Mathijs de Jong (V2B)

In [147]:
import pandas as pd
import numpy as np
from scipy import stats
from cleaning import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


### Importeren dataset

In [ ]:
# Load the data
df = load_data('sap_storing_data_hu_subset.csv')

# Data understanding

In [157]:
df.describe()

,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_sap_meld_ddt,stm_geo_mld,stm_geo_mld_uit_functiepl,stm_equipm_nr_mld,stm_km_van_mld,stm_km_tot_mld,stm_prioriteit,...,stm_progfh_in_duur,stm_progfh_gw_lwd_datum,stm_progfh_gw_lwd_tijd,stm_progfh_gw_duur,stm_progfh_gw_teller,stm_fh_duur,stm_reactie_duur,totale_functiehersteltijd,baseline_fh_duur,difference
count,59.000000,5.900000e+01,5.900000e+01,59,59.000000,59.000000,2.400000e+01,59.000000,59.000000,59.000000,...,5.900000e+01,0.0,0.0,59.0,59.0,59.000000,5.900000e+01,59,59.000000,59.000000
mean,442854.355932,7.028109e+07,2.913219e+05,2011-05-08 06:02:00.186440448,346.440678,346.440678,1.048023e+07,17841.152542,4645.576271,5.796610,...,4.576279e+07,NaN,NaN,99999999.0,0.0,74.389831,2.400481e+08,0 days 01:15:55.016949152,74.610169,-0.220339
min,2.000000,5.005321e+07,0.000000e+00,2006-01-01 20:00:00,1.000000,1.000000,1.001941e+07,0.000000,0.000000,2.000000,...,0.000000e+00,NaN,NaN,99999999.0,0.0,6.000000,0.000000e+00,0 days 00:06:00,12.000000,-465.000000
25%,204.500000,5.005358e+07,0.000000e+00,2006-01-10 09:07:00,88.500000,88.500000,1.017819e+07,0.000000,0.000000,2.000000,...,3.500000e+01,NaN,NaN,99999999.0,0.0,24.000000,4.700000e+01,0 days 00:25:00,24.000000,0.000000
50%,640404.000000,8.076762e+07,0.000000e+00,2013-05-11 09:21:00,203.000000,203.000000,1.039671e+07,0.000000,0.000000,5.000000,...,1.200000e+02,NaN,NaN,99999999.0,0.0,46.000000,1.000000e+08,0 days 00:48:00,32.000000,0.000000
75%,824208.500000,8.087338e+07,4.250000e+01,2014-06-09 04:53:00,594.500000,594.500000,1.092557e+07,24500.000000,0.000000,9.000000,...,1.000000e+08,NaN,NaN,99999999.0,0.0,79.500000,1.000000e+08,0 days 01:21:30,47.000000,36.500000
max,908629.000000,1.000000e+08,4.029661e+06,2016-12-11 17:17:50,950.000000,950.000000,1.101464e+07,102255.000000,46509.000000,9.000000,...,1.000000e+08,NaN,NaN,99999999.0,0.0,475.000000,1.060288e+09,0 days 07:54:50,475.000000,305.000000
std,392324.776085,1.849097e+07,9.887396e+05,NaN,285.024758,285.024758,3.711827e+05,32050.152992,13577.980529,2.911213,...,5.024771e+07,NaN,NaN,0.0,0.0,100.007589,3.986151e+08,0 days 01:39:29.532369173,124.234347,122.985571


# Data cleaning

In [148]:
# Define the columns to drop
columns_to_drop = ['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld',
                   'stm_scenario_mon', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd',
                   'stm_aanntpl_ddt', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt',
                   'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz',
                   'stm_veroorz_groep', 'stm_veroorz_code', 'stm_veroorz_tekst_kort', 'stm_effect', 'stm_afspr_aanvangddt',
                   'stm_mon_eind_ddt', 'stm_mon_vhdsincident', 'stm_dir_betrok_tr', 'stm_aangelegd_dd', 'stm_aangelegd_tijd',
                   'stm_mon_begindatum', 'stm_mon_begintijd', 'stm_progfh_gw_datum', 'stm_mon_eind_datum', 'stm_mon_eind_tijd',
                   'stm_controle_dd', 'stm_akkoord_mon_toewijz', 'stm_status_sapnaarmon', 'stm_fact_jn', 'stm_akkoord_melding_jn',
                   'stm_afsluit_ddt', 'stm_afsluit_dd', 'stm_afsluit_tijd', 'stm_rec_toegev_ddt', 'stm_hinderwaarde',
                   'stm_actie', 'stm_standplaats', 'stm_status_gebr', 'stm_wbi_nummer', 'stm_projnr', 'stm_historie_toelichting',
                   'stm_schade_verhaalb_jn', 'stm_schadenr', 'stm_schade_status_ga', 'stm_schade_statusdatum', 'stm_relatiervo_vorig',
                   'stm_relatiervo_volgend', 'stm_relatiervo', 'stm_afspr_func_hersteldd', 'stm_afspr_func_hersteltijd',
                   'stm_sorteerveld', 'stm_rapportage_maand', 'stm_rapportage_jaar', 'stm_x_bron_publ_dt', 'stm_x_bron_bestandsnaam',
                   'stm_x_bron_arch_dt', 'stm_x_actueel_ind', 'stm_x_run_id', 'stm_x_bk', 'stm_x_start_sessie_dt', 'stm_x_vervallen_ind']

# Drop the unnecessary columns
df = drop_columns(df, columns_to_drop)

# Clean the data (handle missing values, remove columns with excessive NaNs)
#df, avg_list, mode_list = clean_data(df)

# Filter de data op rijen met een totale_functiehersteltijd van tussen de 5 mins en 8 uur.
df = filter_data(df)

# Save the cleaned data to a new CSV (optional)
save_data(df, 'final_db_cleaned.csv')

# Display results (optional)
#print("Data cleaning complete.")
#print("Columns filled with averages: ", avg_list)
#print("Columns filled with mode: ", mode_list)
#print("Cleaned dataframe shape: ", df.shape)


In [149]:
# Tijdelijk
df[['stm_sap_meld_ddt', 'stm_fh_ddt', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_invoer_dat', 'totale_functiehersteltijd']].sample(10)
df['stm_progfh_in_invoer_tijd'].dtype

dtype('O')

# Baseline model

In [151]:
# Updated function to handle both scalar and array return types
def mode_per_group(group):
    mode_result = stats.mode(group, keepdims=True)  # Ensure it returns in an array-like format
    return mode_result.mode[0]  # Safely access the mode value

# Calculate baseline (mode) for each oorzaak code
baseline_modes = df.groupby('stm_oorz_code')['stm_fh_duur'].apply(mode_per_group).reset_index()
baseline_modes.columns = ['stm_oorz_code', 'baseline_fh_duur']

# Merge baseline values with the original DataFrame
df = df.merge(baseline_modes, on='stm_oorz_code')

# Calculate the difference between actual time and baseline (mode)
df['difference'] = df['stm_fh_duur'] - df['baseline_fh_duur']

In [152]:
# Mean Absolute Error (MAE)
mae = df['difference'].abs().mean()
print(f'Mean Absolute Error (MAE): {mae}')


Mean Absolute Error (MAE): 52.52542372881356


In [153]:
# Check if the baseline prediction matches the actual time
df['correct_prediction'] = df['stm_fh_duur'] == df['baseline_fh_duur']

# Calculate the accuracy in percentage
accuracy = df['correct_prediction'].mean() * 100

# Print the accuracy
print(f'Accuracy of the baseline model: {accuracy:.2f}%')


Accuracy of the baseline model: 38.98%


In [154]:
# Onafhankelijke variabelen (X) en afhankelijke variabele (y)
X = df[['stm_progfh_in_duur', 'stm_oorz_code']]
y = df['stm_fh_duur']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineair regressiemodel aanmaken
model = LinearRegression()

# Train het model
model.fit(X_train, y_train)

# Maak voorspellingen op de testset
y_pred = model.predict(X_test)

# Bereken de evaluatiestatistieken
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Zet R2 om in een percentage
r2_percentage = r2 * 100

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2_percentage:.2f}%')


Mean Squared Error: 4430.315235328863
R-squared: -142.58%
